# Basic Data Modeling and Extraction from APIs

In this notebook, we will learn how to fetch data from a public API, JSONPlaceholder, and store it in an SQLite database. We will also practice SQL queries to manipulate and retrieve data.

### Concepts Covered:
- Fetching data from an API using `requests` library
- Creating SQLite tables
- Inserting data into tables
- Querying the data using SQL commands

## Step 1: Setup the SQLite Database
We will first create the necessary tables that match the JSONPlaceholder data schema.

In [ ]:
import sqlite3
import pandas as pd
from IPython.display import display
import uuid

# Create a new SQLite database
conn = sqlite3.connect(str(uuid.uuid4()) + '.db')
cursor = conn.cursor()

# Create tables
create_users_table = '''
CREATE TABLE IF NOT EXISTS users (
    id INTEGER PRIMARY KEY,
    name TEXT,
    username TEXT,
    email TEXT,
    address TEXT,
    phone INTEGER,
    website TEXT,
    company TEXT
);
'''

create_posts_table = '''
CREATE TABLE IF NOT EXISTS posts (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    body TEXT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''

create_comments_table = '''
CREATE TABLE IF NOT EXISTS comments (
    id INTEGER PRIMARY KEY,
    post_id INTEGER,
    name TEXT,
    email TEXT,
    body TEXT,
    FOREIGN KEY (post_id) REFERENCES posts(id)
);
'''

create_albums_table = '''
CREATE TABLE IF NOT EXISTS albums (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''

create_photos_table = '''
CREATE TABLE IF NOT EXISTS photos (
    id INTEGER PRIMARY KEY,
    album_id INTEGER,
    title TEXT,
    url TEXT,
    thumbnail_url TEXT,
    FOREIGN KEY (album_id) REFERENCES albums(id)
);
'''

create_todos_table = '''
CREATE TABLE IF NOT EXISTS todos (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    title TEXT,
    completed BOOLEAN,
    FOREIGN KEY (user_id) REFERENCES users(id)
);
'''

# Execute the SQL commands
cursor.execute(create_users_table)
cursor.execute(create_posts_table)
cursor.execute(create_comments_table)
cursor.execute(create_albums_table)
cursor.execute(create_photos_table)
cursor.execute(create_todos_table)

# Commit the changes
conn.commit()

# Verify the tables were created
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
tables

[('users',), ('posts',), ('comments',), ('albums',), ('photos',), ('todos',)]

## Step 2: Fetch Data from JSONPlaceholder API
We will use the `requests` library to fetch data from the API for each entity: users, posts, comments, albums, photos, and todos.

In [ ]:
import requests


# Define function to fetch data from API
def fetch_data(endpoint):
    url = f'https://jsonplaceholder.typicode.com/{endpoint}'
    response = requests.get(url)
    return response.json()

# Fetch data
users_data = fetch_data('users')
posts_data = fetch_data('posts')
comments_data = fetch_data('comments')
albums_data = fetch_data('albums')
photos_data = fetch_data('photos')
todos_data = fetch_data('todos')

# Print example data
users_data[:2], posts_data[:2], comments_data[:2]

([{'id': 1,
   'name': 'Leanne Graham',
   'username': 'Bret',
   'email': 'Sincere@april.biz',
   'address': {'street': 'Kulas Light',
    'suite': 'Apt. 556',
    'city': 'Gwenborough',
    'zipcode': '92998-3874',
    'geo': {'lat': '-37.3159', 'lng': '81.1496'}},
   'phone': '1-770-736-8031 x56442',
   'website': 'hildegard.org',
   'company': {'name': 'Romaguera-Crona',
    'catchPhrase': 'Multi-layered client-server neural-net',
    'bs': 'harness real-time e-markets'}},
  {'id': 2,
   'name': 'Ervin Howell',
   'username': 'Antonette',
   'email': 'Shanna@melissa.tv',
   'address': {'street': 'Victor Plains',
    'suite': 'Suite 879',
    'city': 'Wisokyburgh',
    'zipcode': '90566-7771',
    'geo': {'lat': '-43.9509', 'lng': '-34.4618'}},
   'phone': '010-692-6593 x09125',
   'website': 'anastasia.net',
   'company': {'name': 'Deckow-Crist',
    'catchPhrase': 'Proactive didactic contingency',
    'bs': 'synergize scalable supply-chains'}}],
 [{'userId': 1,
   'id': 1,
   'tit

## Step 3: Insert Data into SQLite Tables
Now we will insert the fetched data into their corresponding SQLite tables.

In [ ]:
# Insert users data into the users table
def insert_users(data):
    for user in data:
        cursor.execute("""
        INSERT OR REPLACE INTO users (id, name, username, email, address, phone, website, company)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
        """, (user['id'], user['name'], user['username'], user['email'],
              str(user['address']), user['phone'], user['website'], str(user['company'])))
    conn.commit()

# Repeat similar for other tables
insert_users(users_data)
cursor.execute('SELECT * FROM users LIMIT 5;')
cursor.fetchall()

# Insert posts data into posts table
def insert_posts(data):
    for post in data:
        cursor.execute("""
            INSERT OR REPLACE INTO posts (id, user_id, title, body)
            VALUES (?, ?, ?, ?)
        """, (post['id'], post['userId'], post['title'], post['body']))
    conn.commit()

insert_posts(posts_data)
print("Posts data inserted successfully.")
cursor.execute('SELECT * FROM posts LIMIT 5;')
cursor.fetchall()

# Insert comments data into comments table
def insert_comments(data):
    for comment in data:
        cursor.execute("""
            INSERT OR REPLACE INTO comments (id, post_id, name, email, body)
            VALUES (?, ?, ?, ?, ?)
        """, (comment['id'], comment['postId'], comment['name'], comment['email'], comment['body']))
    conn.commit()

insert_comments(comments_data)
print("Comments data inserted successfully.")

# Insert albums data into albums table
def insert_albums(data):
    for album in data:
        cursor.execute("""
            INSERT OR REPLACE INTO albums (id, user_id, title)
            VALUES (?, ?, ?)
        """, (album['id'], album['userId'], album['title']))
    conn.commit()

insert_albums(albums_data)
print("Albums data inserted successfully.")

# Insert photos data into photos table
def insert_photos(data):
    for photo in data:
        cursor.execute("""
            INSERT OR REPLACE INTO photos (id, album_id, title, url, thumbnail_url)
            VALUES (?, ?, ?, ?, ?)
        """, (photo['id'], photo['albumId'], photo['title'], photo['url'], photo['thumbnailUrl']))
    conn.commit()
insert_photos(photos_data)
print("Photos data inserted successfully.")

# insert todos data into todo_tables
def insert_todos(data):
  for todo in data:
    cursor.execute("""
    INSERT OR REPLACE INTO todos (id, user_id, title, completed)
    VALUES (?, ?, ?, ?)
    """, (todo['id'], todo['userId'], todo['title'], todo['completed']))
    conn.commit()

insert_todos(todos_data)
print("Todos data inserted successfully.")

Posts data inserted successfully.
Comments data inserted successfully.
Albums data inserted successfully.
Photos data inserted successfully.
Todos data inserted successfully.


In [ ]:
cursor.execute('''SELECT * FROM users LIMIT 5''')
cursor.fetchall()
#conn.commit()

[(1,
  'Leanne Graham',
  'Bret',
  'Sincere@april.biz',
  "{'street': 'Kulas Light', 'suite': 'Apt. 556', 'city': 'Gwenborough', 'zipcode': '92998-3874', 'geo': {'lat': '-37.3159', 'lng': '81.1496'}}",
  '1-770-736-8031 x56442',
  'hildegard.org',
  "{'name': 'Romaguera-Crona', 'catchPhrase': 'Multi-layered client-server neural-net', 'bs': 'harness real-time e-markets'}"),
 (2,
  'Ervin Howell',
  'Antonette',
  'Shanna@melissa.tv',
  "{'street': 'Victor Plains', 'suite': 'Suite 879', 'city': 'Wisokyburgh', 'zipcode': '90566-7771', 'geo': {'lat': '-43.9509', 'lng': '-34.4618'}}",
  '010-692-6593 x09125',
  'anastasia.net',
  "{'name': 'Deckow-Crist', 'catchPhrase': 'Proactive didactic contingency', 'bs': 'synergize scalable supply-chains'}"),
 (3,
  'Clementine Bauch',
  'Samantha',
  'Nathan@yesenia.net',
  "{'street': 'Douglas Extension', 'suite': 'Suite 847', 'city': 'McKenziehaven', 'zipcode': '59590-4157', 'geo': {'lat': '-68.6102', 'lng': '-47.0653'}}",
  '1-463-123-4447',
  'ra

## Step 4: Query Data from SQLite Tables
Now let's run some SQL queries to retrieve the data we inserted.

In [ ]:
# Example: Fetch posts made by user with ID 1
cursor.execute("SELECT count(*) FROM posts where user_id = 1")
user_posts = cursor.fetchall()
user_posts

[(10,)]

In [ ]:
# Example: Fetch comments on post with ID 1
cursor.execute("SELECT * FROM comments WHERE post_id = 1;")
post_comments = cursor.fetchall()
post_comments

[(1,
  1,
  'id labore ex et quam laborum',
  'Eliseo@gardner.biz',
  'laudantium enim quasi est quidem magnam voluptate ipsam eos\ntempora quo necessitatibus\ndolor quam autem quasi\nreiciendis et nam sapiente accusantium'),
 (2,
  1,
  'quo vero reiciendis velit similique earum',
  'Jayne_Kuhic@sydney.com',
  'est natus enim nihil est dolore omnis voluptatem numquam\net omnis occaecati quod ullam at\nvoluptatem error expedita pariatur\nnihil sint nostrum voluptatem reiciendis et'),
 (3,
  1,
  'odio adipisci rerum aut animi',
  'Nikita@garfield.biz',
  'quia molestiae reprehenderit quasi aspernatur\naut expedita occaecati aliquam eveniet laudantium\nomnis quibusdam delectus saepe quia accusamus maiores nam est\ncum et ducimus et vero voluptates excepturi deleniti ratione'),
 (4,
  1,
  'alias odio sit',
  'Lew@alysha.tv',
  'non et atque\noccaecati deserunt quas accusantium unde odit nobis qui voluptatem\nquia voluptas consequuntur itaque dolor\net qui rerum deleniti ut occaecati'),


## Step 5: Close the Connection
Always close the connection when done.

In [ ]:
# # Close the connection
# cursor.close()
# #conn.close()


# SQL Exercises: Complex Joins and Aggregate Functions

Exercise 1: Count the Total Number of Posts per User

Write a query to count how many posts each user has created. This involves joining the users table with the posts table and using the COUNT() aggregate function.

In [ ]:
cursor.execute("""
""")
result = cursor.fetchall()
result

[(1, 'Leanne Graham', 10),
 (2, 'Ervin Howell', 10),
 (3, 'Clementine Bauch', 10),
 (4, 'Patricia Lebsack', 10),
 (5, 'Chelsey Dietrich', 10),
 (6, 'Mrs. Dennis Schulist', 10),
 (7, 'Kurtis Weissnat', 10),
 (8, 'Nicholas Runolfsdottir V', 10),
 (9, 'Glenna Reichert', 10),
 (10, 'Clementina DuBuque', 10)]

Exercise 2: Retrieve Users with the Most Comments on Their Posts

Write a query to determine which users have received the most comments on their posts. This involves joining the users, posts, and comments tables and grouping by user.

In [ ]:
cursor.execute("""
""")
result = cursor.fetchall()
result

[(1, 'Leanne Graham', 50),
 (2, 'Ervin Howell', 50),
 (3, 'Clementine Bauch', 50),
 (4, 'Patricia Lebsack', 50),
 (5, 'Chelsey Dietrich', 50),
 (6, 'Mrs. Dennis Schulist', 50),
 (7, 'Kurtis Weissnat', 50),
 (8, 'Nicholas Runolfsdottir V', 50),
 (9, 'Glenna Reichert', 50),
 (10, 'Clementina DuBuque', 50)]

Exercise 3: List the Top 3 Most Commented Posts

Write a query to find out which posts have received the most comments. Join the posts and comments tables and display the top 3 results.


In [ ]:
cursor.execute("""
""")
result = cursor.fetchall()
result

[(1,
  'sunt aut facere repellat provident occaecati excepturi optio reprehenderit',
  5),
 (2, 'qui est esse', 5),
 (3, 'ea molestias quasi exercitationem repellat qui ipsa sit aut', 5)]

Exercise 4: Find Albums that Have More than 10 Photos

Write a query to count how many photos are associated with each album and return only those albums that contain more than 10 photos.


In [ ]:
cursor.execute("""
""")
result = cursor.fetchall()
result

[(1, 'quidem molestiae enim', 50),
 (2, 'sunt qui excepturi placeat culpa', 50),
 (3, 'omnis laborum odio', 50),
 (4, 'non esse culpa molestiae omnis sed optio', 50),
 (5, 'eaque aut omnis a', 50),
 (6, 'natus impedit quibusdam illo est', 50),
 (7, 'quibusdam autem aliquid et et quia', 50),
 (8, 'qui fuga est a eum', 50),
 (9, 'saepe unde necessitatibus rem', 50),
 (10, 'distinctio laborum qui', 50),
 (11, 'quam nostrum impedit mollitia quod et dolor', 50),
 (12, 'consequatur autem doloribus natus consectetur', 50),
 (13, 'ab rerum non rerum consequatur ut ea unde', 50),
 (14, 'ducimus molestias eos animi atque nihil', 50),
 (15, 'ut pariatur rerum ipsum natus repellendus praesentium', 50),
 (16, 'voluptatem aut maxime inventore autem magnam atque repellat', 50),
 (17, 'aut minima voluptatem ut velit', 50),
 (18, 'nesciunt quia et doloremque', 50),
 (19, 'velit pariatur quaerat similique libero omnis quia', 50),
 (20, 'voluptas rerum iure ut enim', 50),
 (21, 'repudiandae voluptatem op

Exercise 5: Find Users with Incomplete Todos

Write a query to retrieve users who have at least one incomplete to-do item. Join the users and todos tables and use the WHERE clause to filter based on the completed field.

In [ ]:
cursor.execute("""

""")
result = cursor.fetchall()
result

[(1, 'Leanne Graham', 9),
 (2, 'Ervin Howell', 12),
 (3, 'Clementine Bauch', 13),
 (4, 'Patricia Lebsack', 14),
 (5, 'Chelsey Dietrich', 8),
 (6, 'Mrs. Dennis Schulist', 14),
 (7, 'Kurtis Weissnat', 11),
 (8, 'Nicholas Runolfsdottir V', 9),
 (9, 'Glenna Reichert', 12),
 (10, 'Clementina DuBuque', 8)]

Exercise 6: Calculate the Average Number of Comments per Post

Write a query to calculate the average number of comments per post. This involves joining the posts and comments tables and using the AVG() function.


In [ ]:
cursor.execute("""
""")
result = cursor.fetchall()
result

[(1, 1, 3.0),
 (2, 2, 8.0),
 (3, 3, 13.0),
 (4, 4, 18.0),
 (5, 5, 23.0),
 (6, 6, 28.0),
 (7, 7, 33.0),
 (8, 8, 38.0),
 (9, 9, 43.0),
 (10, 10, 48.0),
 (11, 11, 53.0),
 (12, 12, 58.0),
 (13, 13, 63.0),
 (14, 14, 68.0),
 (15, 15, 73.0),
 (16, 16, 78.0),
 (17, 17, 83.0),
 (18, 18, 88.0),
 (19, 19, 93.0),
 (20, 20, 98.0),
 (21, 21, 103.0),
 (22, 22, 108.0),
 (23, 23, 113.0),
 (24, 24, 118.0),
 (25, 25, 123.0),
 (26, 26, 128.0),
 (27, 27, 133.0),
 (28, 28, 138.0),
 (29, 29, 143.0),
 (30, 30, 148.0),
 (31, 31, 153.0),
 (32, 32, 158.0),
 (33, 33, 163.0),
 (34, 34, 168.0),
 (35, 35, 173.0),
 (36, 36, 178.0),
 (37, 37, 183.0),
 (38, 38, 188.0),
 (39, 39, 193.0),
 (40, 40, 198.0),
 (41, 41, 203.0),
 (42, 42, 208.0),
 (43, 43, 213.0),
 (44, 44, 218.0),
 (45, 45, 223.0),
 (46, 46, 228.0),
 (47, 47, 233.0),
 (48, 48, 238.0),
 (49, 49, 243.0),
 (50, 50, 248.0),
 (51, 51, 253.0),
 (52, 52, 258.0),
 (53, 53, 263.0),
 (54, 54, 268.0),
 (55, 55, 273.0),
 (56, 56, 278.0),
 (57, 57, 283.0),
 (58, 58, 288.

Exercise 7: Retrieve Users Who Have Both Posts and Todos

Write a query to retrieve users who have created at least one post and one to-do item. You can use subqueries or a combination of JOIN and INTERSECT.

In [ ]:
cursor.execute("""
""")
result = cursor.fetchall()
result

[(1, 'Leanne Graham', 200, 200),
 (2, 'Ervin Howell', 200, 200),
 (3, 'Clementine Bauch', 200, 200),
 (4, 'Patricia Lebsack', 200, 200),
 (5, 'Chelsey Dietrich', 200, 200),
 (6, 'Mrs. Dennis Schulist', 200, 200),
 (7, 'Kurtis Weissnat', 200, 200),
 (8, 'Nicholas Runolfsdottir V', 200, 200),
 (9, 'Glenna Reichert', 200, 200),
 (10, 'Clementina DuBuque', 200, 200)]

In [ ]:
# Close the connection
cursor.close()
conn.close()
